In [25]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd

In [26]:
import h5py
from scipy.io import savemat

### Load Original Data

In [27]:
nx = 256
ny = 128
nz = 160

In [28]:
filename = "/Users/Alex/Desktop/MEng_Resources/2D_3D_DNS_Data/snapshots.pkl"

In [29]:
with open(filename, 'rb') as f:
    obj = pickle.load(f)
    uvw3D_field = obj

In [30]:
uvw3D_field.shape

(100, 256, 128, 160, 3)

### Load Trained Model

In [31]:
model = tf.keras.models.load_model("/Users/Alex/Desktop/MEng_Resources/Project_Run_Logbook/Full_100_Run/Run_2/Test_Model")

# model = tf.keras.models.load_model("/Users/Alex/Desktop/MEng_Resources/Project_Run_Logbook/100_dt5_Run/Test_Model")

### Predict 4D Dataset

In [32]:
# Define empty 2D dataset of 5 sections to predict 3D field
test_sections = np.empty([nx, ny, 15])
dt = 0 # in snapshots
snapshots = np.arange(2*dt,100-2*dt, dtype='int')

In [33]:
snapshots.shape

(100,)

In [34]:
slice_pos = np.array([(nz - 1) * 0.1, (nz - 1) * 0.3, (nz - 1) * 0.5, (nz - 1) * 0.7, (nz - 1) * 0.9])
slice_pos = slice_pos.astype(int)

In [35]:
predictions = np.empty([len(snapshots),nx,ny,nz,3])

In [36]:
predictions.shape

(100, 256, 128, 160, 3)

In [37]:
for ref_snap in snapshots:
    test_sections = np.empty([nx, ny, 15])
    test_sections[:, :, 0:3] = uvw3D_field[ref_snap - 2*dt, :, :, slice_pos[0], :]
    test_sections[:, :, 3:6] = uvw3D_field[ref_snap - dt, :, :, slice_pos[1], :]
    test_sections[:, :, 6:9] = uvw3D_field[ref_snap, :, :, slice_pos[2], :]
    test_sections[:, :, 9:12] = uvw3D_field[ref_snap + dt, :, :, slice_pos[3], :]
    test_sections[:, :, 12:] = uvw3D_field[ref_snap + 2*dt, :, :, slice_pos[4], :]
    
    test_sections = np.expand_dims(test_sections, axis=0)
    
    predictions[ref_snap - 2*dt,:,:,:,:] = model.predict(test_sections, batch_size=1)

In [38]:
predictions.shape

(100, 256, 128, 160, 3)

### Calculate Ensemble L2 Error Norm

In [18]:
true_field = uvw3D_field

In [19]:
true_field_zero_mean = np.zeros(true_field.shape)
#Subtract mean from each component
true_field_zero_mean[:,:,:,:,0] = true_field[:,:,:,:,0] - np.mean(true_field[:,:,:,:,0])
true_field_zero_mean[:,:,:,:,1] = true_field[:,:,:,:,1] - np.mean(true_field[:,:,:,:,1])
true_field_zero_mean[:,:,:,:,2] = true_field[:,:,:,:,2] - np.mean(true_field[:,:,:,:,2])

In [21]:
pred_flat = np.ravel(predictions)
true_flat = np.ravel(true_field)
true_flat_zero_mean = np.ravel(true_field_zero_mean)
error = true_flat - pred_flat

In [ ]:
L2 = np.linalg.norm(error) / np.linalg.norm(true_flat_zero_mean)

In [ ]:
L2

### Save File for POD Analaysis

In [44]:
filename = 'pod/v_pred_data_dt0'
data = predictions[:,:,:,83,1]

In [45]:
data.shape

(100, 256, 128)

In [46]:
hf = h5py.File(filename, 'w')

In [47]:
hf.create_dataset('data', data=data)

<HDF5 dataset "data": shape (100, 256, 128), type "<f8">

In [48]:
hf.close()